# リカレントニューラルネットワーク

最初の時間ステップt=0では、隠れユニットがそれぞれ0または小さな乱数で初期化される。

## 長期的な相互作用の学習
BPTT(Backpropagation Through Time)により、勾配消失問題や勾配発散問題が発生する。解決策として、
* 勾配刈り込み（勾配の閾値を設定）
* T-BPTT(Truncated Backpropagation Through Time)（逆伝搬できる時間ステップの数を制限する）
* 長短期記憶(Long Short-Term Memory:LSTM)

## LSTM

LSTM（Long Short-Term Memory）の数式の根拠は、「**情報を長期的に保持し、必要なときに取り出す**」という目的のもと、**勾配消失問題（vanishing gradient）を回避する構造**を持つことにあります。以下に、LSTMの数式とそれぞれの根拠を解説します。

---

### LSTMの基本構造と数式

時間ステップ $t$ におけるLSTMの更新式は次のとおりです（入力 $\mathbf{x}_t$、前の隠れ状態 $\mathbf{h}_{t-1}$、セル状態 $\mathbf{c}_{t-1}$）：

$$
\begin{aligned}
\mathbf{i}_t &= \sigma(\mathbf{W}_i \mathbf{x}_t + \mathbf{U}_i \mathbf{h}_{t-1} + \mathbf{b}_i) \quad &\text{（入力ゲート）}\\
\mathbf{f}_t &= \sigma(\mathbf{W}_f \mathbf{x}_t + \mathbf{U}_f \mathbf{h}_{t-1} + \mathbf{b}_f) \quad &\text{（忘却ゲート）}\\
\mathbf{o}_t &= \sigma(\mathbf{W}_o \mathbf{x}_t + \mathbf{U}_o \mathbf{h}_{t-1} + \mathbf{b}_o) \quad &\text{（出力ゲート）}\\
\tilde{\mathbf{c}}_t &= \tanh(\mathbf{W}_c \mathbf{x}_t + \mathbf{U}_c \mathbf{h}_{t-1} + \mathbf{b}_c) \quad &\text{（候補セル状態）}\\
\mathbf{c}_t &= \mathbf{f}_t \odot \mathbf{c}_{t-1} + \mathbf{i}_t \odot \tilde{\mathbf{c}}_t \quad &\text{（セル状態の更新）}\\
\mathbf{h}_t &= \mathbf{o}_t \odot \tanh(\mathbf{c}_t) \quad &\text{（出力）}
\end{aligned}
$$

---

### 数式の根拠と設計思想

| 数式                           | 意図・根拠                                                 |
| ---------------------------- | ----------------------------------------------------- |
| $\mathbf{i}_t$（入力ゲート）        | 新しい情報をどれだけセルに加えるか制御する。重要な入力だけ通す。                      |
| $\mathbf{f}_t$（忘却ゲート）        | セルに保持された情報をどれだけ忘れるかを決める。長期依存を制御。<br>→ **勾配消失の回避**にも貢献 |
| $\tilde{\mathbf{c}}_t$（候補セル） | 新しく加えたい候補情報。tanhを使って情報のスケーリング（\[-1,1]に制限）             |
| $\mathbf{c}_t$（セル状態）         | **LSTMの核心**。時間を超えて情報を伝える経路。加算により勾配消失を防ぐ               |
| $\mathbf{o}_t$（出力ゲート）        | セルからどれだけ外に出力するか決める。必要な情報だけ抽出                          |
| $\mathbf{h}_t$（出力）           | 実際に次のRNNステップや出力に使われる値                                 |

---

### 根拠まとめ

1. **勾配消失を避ける設計**

   * セル状態 $\mathbf{c}_t$ は加算で更新されるため、時間方向に勾配が直接流れやすい（BPTTでの安定性）。
   * これは従来のRNN（$\mathbf{h}_t = \tanh(Wx_t + Uh_{t-1})$）では難しかった。

2. **学習可能なメモリ制御**

   * 忘却・入力・出力ゲートがそれぞれ独立に学習されるため、文脈依存の動的な制御が可能。

3. **ゲート構造による柔軟な情報選択**

   * 重要な情報のみを通し、不要な情報を遮断・忘却できる。

---

### 関連論文と歴史的背景

* 初出論文：**Hochreiter and Schmidhuber, 1997**, ["Long Short-Term Memory"](https://www.bioinf.jku.at/publications/older/2604.pdf)
* 背景：従来のRNNでは長期依存がうまく学習できなかった（勾配消失・爆発）。それに対する解決策として提案。
